In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

DIR_DATA = 'Final_dataset_small2'
DIR_MODEL = 'models/xception_fine_no_freeze_flatten.h5'
classes = os.listdir(DIR_DATA)

In [3]:
from tensorflow import keras

from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear

replace2linear = ReplaceToLinear()
model = keras.models.load_model(DIR_MODEL)

saliency = Saliency(model,
                    model_modifier=replace2linear)

In [4]:
def gaussian_pdf(x, mean=0.5, std_dev=0.15):
    return (1.0 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

for class_index, class_ in enumerate(classes[2:]):
    class_index += 2
    imagenes = [d for d in os.listdir(os.path.join(DIR_DATA, class_)) if d.endswith('.jpg')]
    for img in imagenes:
        img_array = cv2.imread(os.path.join(DIR_DATA, class_, img)) / 255.0
        saliency_map = saliency(CategoricalScore(class_index), img_array, smooth_samples=30, smooth_noise=0.2, normalize_map = False)[0]
        saliency_map = (saliency_map - np.min(saliency_map)) / (np.max(saliency_map) - np.min(saliency_map))
        if not os.path.exists(f'saliency_maps/{class_}'):
                os.makedirs(f'saliency_maps/{class_}')
        cv2.imwrite(f'saliency_maps/{class_}/{img[:-4]}.png', saliency_map * 255.0)
      

In [5]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")
sam.to(device="cuda")

for class_index, class_ in enumerate(classes[2:]):
    class_index += 2
    print(f'{class_index} - {class_}')
    imagenes = os.listdir(os.path.join(DIR_DATA, class_))
    for j, img in enumerate(imagenes):
        if not img.endswith('.jpg'): continue
        try:
            img_array = cv2.imread(os.path.join(DIR_DATA, class_, img))

            mask_generator = SamAutomaticMaskGenerator(sam, stability_score_thresh = 0.7)
            masks_results = mask_generator.generate(img_array)
            masks = [
                mask['segmentation']
                for mask
                in sorted(masks_results, key=lambda x: x['area'], reverse=True) if mask['area'] > 300
            ]

            # Get mask with the highest score compared to the heatmap
            valid_masks = []

            max_area = np.max([np.sum(mask) for mask in masks])

            # Read the saliency map
            saliency_map = cv2.imread(f'saliency_maps/{class_}/{img[:-4]}.png', cv2.IMREAD_GRAYSCALE)
            saliency_map = saliency_map.astype(np.float32) / 255.0
            saliency_map_mean = np.mean(saliency_map)
            for mask in masks:
                area = np.sum(mask)
                mask_ = mask.astype(np.float32)
                tmp = np.zeros((img_array.shape[0], img_array.shape[1])).astype(np.float32)
                tmp = saliency_map
                tmp[saliency_map < saliency_map_mean * 0.5] = -2
                valor = np.sum((mask_ * tmp))
                valid_masks.append((mask, valor))

            valid_masks = sorted(valid_masks, key=lambda x: x[1], reverse=True)[0]

            mask = valid_masks[0]

            # Save mask
            if not os.path.exists(f'masks/{class_}'):
                os.makedirs(f'masks/{class_}')
            cv2.imwrite(f'masks/{class_}/{img[:-4]}.png', mask * 255)
        except Exception as e:
            print(e)
            print(f'Error en {class_}/{img}')
            continue

2 - Marron
3 - Medicamento
4 - Pilas
5 - Punto
6 - RAEE
7 - Resta
8 - Ropa
9 - Verde
